<a href="https://colab.research.google.com/github/erendagasan/Eren-Dagasan-Personal/blob/main/DAMPFER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [39]:
life_time_final = 60 * 60 * 24 * 30 * 12 * 50
time_step = 300
Pinitial = 2000
Pambient = 1000
V = 0.001
Pcritic = 1500
Qinitial_vals = np.arange(1.1E-9, 1E-8, 1e-9)
time_actual = np.arange(time_step, life_time_final, time_step)

df_list = []

for Qinitial in Qinitial_vals[:1]:
    # Defining starting variables
    Premain_start = Pinitial - (Qinitial * time_step / V)
    Q_start = (Qinitial * ((Premain_start**2) - (Pambient**2)) / ((Pinitial**2) - (Pambient**2)))
    Pambient_start = Pambient+Pinitial-Premain_start

    # Premain_list = []
    # Premain_list.append(Pinitial)
    # Premain_list.append(Premain_start)

    # Pambient_list = []
    # Pambient_list.append(Pambient)
    # Pambient_list.append(Pambient_start)

    # Q_calc_future_list = []
    # Q_calc_future_list.append(Qinitial)
    # Q_calc_future_list.append(Q_start)

    # Premain_calc_buffer = Premain_start
    # Premain_calc_buffer_before = Premain_start
    # Pambient_calc_buffer = Pambient_start

    # Q_buffer_calc = Q_start

    # i = 0
    # while Premain_calc_buffer > Pcritic and Pambient_calc_buffer < Pcritic:
    #     Premain_calc_buffer = Premain_calc_buffer - (Q_buffer_calc * time_step / V)
    #     Premain_list.append(Premain_calc_buffer)

    #     Pambient_calc_buffer = Pambient + (Pinitial-Premain_calc_buffer)
    #     Pambient_list.append(Pambient_calc_buffer)

    #     calc = Q_buffer_calc * ((Premain_calc_buffer**2) - (Pambient**2)) / ((Premain_calc_buffer_before**2) - (Pambient**2))
    #     Q_calc_future_list.append(calc)
    #     Q_buffer_calc = calc

    #     Premain_calc_buffer_before = Premain_calc_buffer

    #     i+=1


    Premain_real_list = []
    Premain_real_list.append(Pinitial)
    Premain_real_list.append(Premain_start)

    Pambient_real_list = []
    Pambient_real_list.append(Pambient)
    Pambient_real_list.append(Pambient_start)

    Q_real_future_list = []
    Q_real_future_list.append(Qinitial)
    Q_real_future_list.append(Q_start)

    Premain_real_buffer = Premain_start
    Premain_real_buffer_before = Premain_start
    Pambient_real_buffer = Pambient_start
    Pambient_real_buffer_before = Pambient_start

    Q_buffer_real = Q_start

    i = 0
    while Premain_real_buffer > Pcritic and Pambient_real_buffer < Pcritic :
        Premain_real_buffer = Premain_real_buffer - (Q_buffer_real * time_step / V)
        Premain_real_list.append(Premain_real_buffer)

        Pambient_real_buffer = Pambient + (Pinitial-Premain_real_buffer)
        Pambient_real_list.append(Pambient_real_buffer)

        calc = Q_buffer_real * ((Premain_real_buffer**2) - (Pambient_real_buffer**2)) / ((Premain_real_buffer_before**2) - (Pambient_real_buffer_before**2))
        Q_real_future_list.append(calc)
        Q_buffer_real = calc

        Premain_real_buffer_before = Premain_real_buffer
        Pambient_real_buffer_before = Pambient_real_buffer

        i+=1

    # Q_diff = np.array(Q_real_future_list)-np.array(Q_calc_future_list)

    # df = pd.DataFrame({'Qcalc': Q_calc_future_list, 'Qreal': Q_real_future_list, 'Qdiff': Q_diff, 'Pinternal': Premain_list, 'Penv': Pambient_list, 'Duration': time_actual[:len(Premain_list)]})

    # #Converting duration in seconds to specified time variables
    # df['Duration_min']   = df['Duration']/60
    # df['Duration_hour']  = df['Duration_min']/60
    # df['Duration_day']   = df['Duration_hour']/24
    # df['Duration_month'] = df['Duration_day']/30
    # df['Duration_year']  = df['Duration_month']/12

    # #Adding all the df's that depends different Qinitial values to a list
    # df_list.append(df)

In [40]:
pd.options.display.float_format = '{:.25e}'.format

In [42]:
data = {
    "Qcalc": Q_calc_future_list,
    "Penv_calc": Pambient_list,
    "Pint_calc": Premain_list,
    "Duration": time_actual[:len(Premain_list)]
}

data['Duration_min']   = data['Duration']/60
data['Duration_hour']  = data['Duration_min']/60
data['Duration_day']   = data['Duration_hour']/24
data['Duration_month'] = data['Duration_day']/30
data['Duration_year']  = data['Duration_month']/12

data = pd.DataFrame(data)
data.tail(10)

,Qcalc,Penv_calc,Pint_calc,Duration,Duration_min,Duration_hour,Duration_day,Duration_month,Duration_year
2321926,4.5833457596753891301726465e-10,1.4999988703330279804504244e+03,1.5000011296669720195495756e+03,696578100,1.1609635000000000000000000e+07,1.9349391666666665696538985e+05,8.0622465277777773735579103e+03,2.6874155092592593518929789e+02,2.2395129243827160081536931e+01
2321927,4.5833442471702184590099064e-10,1.4999990078334008103411179e+03,1.5000009921665991896588821e+03,696578400,1.1609640000000000000000000e+07,1.9349400000000000000000000e+05,8.0622500000000000000000000e+03,2.6874166666666667424578918e+02,2.2395138888888890704720325e+01
2321928,4.5833427346656847169188322e-10,1.4999991453337281654967228e+03,1.5000008546662718345032772e+03,696578700,1.1609645000000000000000000e+07,1.9349408333333334303461015e+05,8.0622534722222226264420897e+03,2.6874178240740741330228047e+02,2.2395148533950617775190040e+01
2321929,4.5833412221617894548630724e-10,1.4999992828340102732909145e+03,1.5000007171659897267090855e+03,696579000,1.1609650000000000000000000e+07,1.9349416666666665696538985e+05,8.0622569444444443433894776e+03,2.6874189814814815235877177e+02,2.2395158179012344845659754e+01
2321930,4.5833397096585331898305097e-10,1.4999994203342469063500175e+03,1.5000005796657530936499825e+03,696579300,1.1609655000000000000000000e+07,1.9349425000000000000000000e+05,8.0622604166666669698315673e+03,2.6874201388888889141526306e+02,2.2395167824074075468843148e+01
2321931,4.5833381971559117859180815e-10,1.4999995578344382920477074e+03,1.5000004421655617079522926e+03,696579600,1.1609660000000000000000000e+07,1.9349433333333334303461015e+05,8.0622638888888895962736569e+03,2.6874212962962963047175435e+02,2.2395177469135802539312863e+01
2321932,4.5833366846539304130046162e-10,1.4999996953345842030103086e+03,1.5000003046654157969896914e+03,696579900,1.1609665000000000000000000e+07,1.9349441666666665696538985e+05,8.0622673611111104037263431e+03,2.6874224537037036952824565e+02,2.2395187114197529609782578e+01
2321933,4.5833351721525880371143481e-10,1.4999998328346846392378211e+03,1.5000001671653153607621789e+03,696580200,1.1609670000000000000000000e+07,1.9349450000000000000000000e+05,8.0622708333333330301684327e+03,2.6874236111111110858473694e+02,2.2395196759259260232965971e+01
2321934,4.5833336596518805223442145e-10,1.4999999703347398281039204e+03,1.5000000296652601718960796e+03,696580500,1.1609675000000000000000000e+07,1.9349458333333334303461015e+05,8.0622743055555556566105224e+03,2.6874247685185184764122823e+02,2.2395206404320987303435686e+01
2321935,4.5833321471518140725488095e-10,1.5000001078347495422349311e+03,1.4999998921652504577650689e+03,696580800,1.1609680000000000000000000e+07,1.9349466666666665696538985e+05,8.0622777777777773735579103e+03,2.6874259259259258669771953e+02,2.2395216049382714373905401e+01


In [14]:
data = {
    "Qreal": Q_real_future_list,
    "Penv_real": Pambient_real_list,
    "Pint_real": Premain_real_list,
    "Duration": time_actual[:len(Premain_real_list)]
}

data = pd.DataFrame(data)

data['Duration_min']   = data['Duration']/60
data['Duration_hour']  = data['Duration_min']/60
data['Duration_day']   = data['Duration_hour']/24
data['Duration_month'] = data['Duration_day']/30
data['Duration_year']  = data['Duration_month']/12

data["Qreal"].iloc[3]

1.0999973380019565e-09